### <center> Assignment 3 Harris Corner Detector & Optical Flow </center>
<center> Computer Vision 1 University of Amsterdam </center>
    <center> Due 23:59, October 1, 2022 (Amsterdam time) </center>

## General Guidelines
Your code and discussion must be completed in this **jupyter notebook** before the deadline by submitting it to the Canvas Lab 3 Assignment. Submit your assignment in a **zip file**, with all the relevant files and images need to run your notebook. Name your zip file as follows:  **StudentID1_StudentID2_StudentID3.zip**
For full credit, make sure your answers follow these guidelines:

- Make sure you use the provided python environment. You can create the environment using conda and the provided YAML file using the following command: `conda env create --file=CV1_env.yaml`, then activate it as `conda activate cv1`. Using different packages versions may result in the impossibility to run the submitted code and therefore in the subtraction of points. Below you will find a code cell to check the versions of your packages. 
- Please express your thoughts concisely. The number of words does not necessarily correlate with how well you understand the concepts.
- Answer all given questions (in **<font color=green >Question** parts). Briefly describe what you implemented. **<font color=blue >Hint** parts are there to give you hints to answer questions.
- Try to understand the problem as much as you can. When answering a question, give evidence (qualitative and/or quantitative results, references to papers, etc.) to support your arguments. Note that not everything might be explicitly asked for and you are expected to think about what might strengthen you arguments and make your notebook self-contained and complete.
- Analyze your results and discuss them, e.g. why algorithm A works better than algorithm B in a certain problem.
- Tables and figures must be accompanied by a brief description. Do not forget to add a number, a title, and if applicable name and unit of variables in a table, name and unit of axes and legends in a figure.
- Make sure all the code in your notebook runs without errors or bugs before submitting. Code that does not run can result in a lower grade. 

**Late submissions** are not allowed. Assignments that are submitted after the strict deadline will not be graded. In case of submission conflicts, TAs’ system clock is taken as reference. We strongly recommend submitting well in advance, to avoid last minute system failure issues.

**Plagiarism note:** Keep in mind that plagiarism (submitted materials which are not your work) is a serious crime and any misconduct shall be punished with the university regulations.

In [ ]:
# python packages needed for this assignment
import numpy as np
import cv2
import matplotlib.pyplot as plt
from scipy import signal
from scipy.ndimage import maximum_filter
from scipy import signal 
import os
import matplotlib
import scipy

In [ ]:
# Make sure you're using the provided environment!
assert cv2.__version__ == "3.4.2", "You're not using the provided Python environment!"
assert np.__version__ == "1.19.5", "You're not using the provided Python environment!"
assert matplotlib.__version__ == "3.3.4", "You're not using the provided Python environment!"
assert scipy.__version__ == "1.7.3", "You're not using the provided Python environment!"
# Proceed to the next cell if you don't get any error.

## 1. Harris Corner Detector (45pts)
In this section, a derivation of the *[Harris Corner Detector](https://www.semanticscholar.org/paper/A-Combined-Corner-and-Edge-Detector-Harris-Stephens/6818668fb895d95861a2eb9673ddc3a41e27b3b3)* is presented.

Given a shift $(\Delta x,\Delta y)$ at a point $(x, y)$, the auto-correlation function is defined as:

$$c(\Delta x,\Delta y) = \sum\limits_{(x,y)\in W(x,y)} {w(x,y)(I(x+\Delta x,y+\Delta y)-I(x,y))^2} \tag {1}$$

where $W(x,y)$ is a window centered at point $(x,y)$ and $w(x,y)$ is a Gaussian function. For simplicity, from now on, $\sum\limits_{(x,y)\in W(x,y)}$ will be referred to as $\sum\limits_{W}$.
Approximating the shifted function by the first-order Taylor expansion we get:

\begin{eqnarray}
I(x+\Delta x,y+ \Delta y) & \approx & I(x,y) + I_x(x,y)\Delta x + I_y(x,y)\Delta y \tag {2} \\
&=& I(x,y) + [I_x(x,y) \ I_y(x,y)] \begin{bmatrix} \Delta x \\ \Delta y \end{bmatrix}, \tag {3}
\end{eqnarray}

where $I_x$ and $I_y$ are partial derivatives of $I(x,y)$. The first gradients can be approximated by: 
\begin{eqnarray}
    I_x &=& \frac{\partial I}{\partial x} \approx I * G_x, \quad G_x = (-1,0,1) \tag {4} \\ 
    I_y &=& \frac{\partial I}{\partial y} \approx I * G_y, \quad G_y = (-1,0,1)^T \tag {5}
\end{eqnarray}


Note that using the kernel $(-1, 1)$ to approximate the gradients is also correct. The auto-correlation function can now be written as:
\begin{eqnarray}
c(\Delta x,\Delta y) &=& \sum\limits_{W} {w(x,y) (I(x+\Delta x,y+\Delta y)-I(x,y))^2} \tag {6} \\
& \approx & \sum\limits_{W} {w(x,y) ([I_x(x,y) \ I_y(x,y)]\begin{bmatrix} \Delta x \\ 
\Delta y \end{bmatrix})^2} \tag {7} \\
&=& [\Delta x \ \Delta y]Q(x,y)\begin{bmatrix} \Delta x \\ \Delta y \end{bmatrix}, \, \tag {8}
\end{eqnarray}

where $Q(x,y)$ is given by:

\begin{eqnarray}
Q(x,y) &=& \sum\limits_{W} w(x,y){\begin{bmatrix} I_x(x,y)^2  & I_x(x,y)I_y(x,y)  \\  I_x(x,y)I_y(x,y)  & I_y(x,y)^2   \end{bmatrix}} \tag {9} \\
&=& \begin{bmatrix} 
\sum\limits_{W}{I_x(x,y)^2} * w(x, y) & \sum\limits_{W}{I_x(x,y)I_y(x,y)} * w(x, y) \\  
\sum\limits_{W}{I_x(x,y)I_y(x,y)} * w(x, y) & \sum\limits_{W}{I_y(x,y)^2} * w(x, y)  \end{bmatrix} \tag {10}\\
&=& \begin{bmatrix} A & B \\  B & C  \end{bmatrix}.\, \tag {11}
\end{eqnarray}

The *cornerness* $H(x,y)$ is defined by the two eigenvalues of $Q(x,y)$, e.g. $\lambda_1$ and $\lambda_2$:

\begin{eqnarray}
\label{eq12}
H &=& \lambda_1 \lambda_2 - 0.04(\lambda_1 + \lambda_2)^2 \tag{12}\\
&=& det(Q) - 0.04(trace(Q))^2 \tag{13}\\
&=& (AC - B^2) - 0.04(A+C)^2. \ \tag{14}
\end{eqnarray}

In this section, you are going to implement $Equation ~\eqref{eq12}$ to calculate $H$ and use it to detect the corners in an image.

**<font color=blue > Hint**: For that purpose, you need to compute the elements of **Q**, i.e. $A$, $B$ and $C$. To do that, you need to calculate $I_x$, which is the smoothed derivative of the image. That can be obtained by convolving the first order Gaussian derivative, $G_d$, with the image $I$ along the x-direction. Then, **A** can be obtained by squaring $I_x$, and then convolving it with a Gaussian, $G$. Similarly, **B** and **C** can be obtained. For example, to get **C**, you need to convolve the image with $G_d$ along the y-direction (to obtain $I_y$), raise it to the square, then convolve it with $G$. 

**<font color=blue > Hint**: The corner points are the local maxima of **H**. Therefore, you should check for every point in $H$, (1) if it is greater than all its neighbours (in an $n \times n$ window centered around this point) and (2) if it is greater than the user-defined threshold. If both conditions are met, then the point is labeled as a corner point.

**<font color=green > Question - 1 (35-*pts*)**
    
1. Create a function to implement the Harris Corner Detector. Your function should return matrix $H$, the indices of rows of the detected corner points **r**, and the indices of columns of those points **c**, where the first corner is given by $(r[0],c[0])$.

    
2. Implement another function that plots three figures: The computed image derivatives $I_x$ and $I_y$, and the original image with the corner points plotted on it. Show your results on example images **toy/00000001.jpg** and **basketball/085.jpg**. Remember to experiment with different threshold values to see the impact on which corners are found.
    
    
3. Is the algorithm rotation-invariant? How about your implementation? Rotate **toy/00000001.jpg** image 45 and 90 degrees and run the Harris Corner Detector algorithm on the rotated images. Explain your answer and support it with your observations.

*Note:* You are allowed to use *scipy.signal.convolve2d* to perform convolution, and *scipy.ndimage.gaussian\_filter* to obtain your image derivatives. 
Include a demo function to run your code.

In [ ]:
# Q1.1 - Type your code here
def calc_derivatives(image):
    # first order derivatives
    G_x = np.array([[-1, 0, 1]])
    G_y = np.array([[-1], [0], [1]])
    
    I_x = signal.convolve2d(image, G_x, mode='same')
    I_y = signal.convolve2d(image, G_y, mode='same')
    
    return I_x, I_y

def harris_corner_detector(I_x, I_y, max_filter_n=5, threshold=5e6):
    A = scipy.ndimage.gaussian_filter(I_x**2, 1)
    B = scipy.ndimage.gaussian_filter(I_x*I_y, 1)
    C = scipy.ndimage.gaussian_filter(I_y**2, 1)
    
    H = (A*C - B**2) - 0.04*(A + C)**2
    
    # we search for the maximum value around each pixel in a 5 by 5 window
    # then use it as a mask for our original H so only the maximum in that 5 by 5 remains.
    maxima = H*(H == maximum_filter(H, size=(max_filter_n,max_filter_n), mode='constant'))
    points = np.argwhere(maxima > threshold)
    
    return H, points

In [ ]:
# Q1.2 - Type your code here
def visualize_corners(image, points, title='', alpha=0.7,save=False,name='',show=True,show_ax = True,quiver=False,point=0,vx_list=0,vy_list=0,folder = ''):
    corners = np.zeros((image.shape[0], image.shape[1], 3))
    '''
    for x, y in points:
        # create a broader area around corner points for better visualization
        corners[x-2:x+2:1, y-2:y+2:1, :] = np.array([1, 0, 0])
    '''
    fig= plt.figure()
    plt.title(title)
    plt.imshow(image, cmap='gray')
    points = np.array(points)
    plt.plot(points[:,1],points[:,0],'rx')
    if quiver:
        plt.quiver(point[:,1],point[:,0],vx_list,vy_list,angles = 'uv',color='white')
    if not show_ax:
        plt.axis('off')
    if save:
        plt.savefig('./Results/' +folder + '/'+ name + '.png')

    if show:
        plt.show()
    
    return fig

img = cv2.imread('data/toy/00000001.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
I_x, I_y = calc_derivatives(img)
_, points = harris_corner_detector(I_x, I_y, threshold=3e6)

visualize_corners(img, points, title='Original image plotted with corner points.')
visualize_corners(I_x, points, title='Derivative in x-direction plotted with corner points.')
visualize_corners(I_y, points, title='Derivative in y-direction image plotted with corner points.')

img = cv2.imread('data/basketball/085.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
I_x, I_y = calc_derivatives(img)
_, points = harris_corner_detector(I_x, I_y, threshold=1.2e7)

visualize_corners(img, points, title='Original image plotted with corner points.')
visualize_corners(I_x, points, title='Derivative in x-direction plotted with corner points.')
visualize_corners(I_y, points, title='Derivative in y-direction image plotted with corner points.')

<b>Answer:</b> After testing with 45 and 90 degrees it can be concluded that Harris Corner Detection (HCD) is indeed rotation invariant. This is also obvious from the method itself. HCD looks at the distribution of image gradients for an image. An edge is defined by a large derivative in a single direction; i.e. an edge in the x-direction will have a large response when taking the derivative in the y-direction. A corner, however, is defined by two edges connecting at a point/angle. The more this angle heads towards 90 degrees, the bigger the spread between the derivatives in the x- and y-direction becomes. 

----------- meer tekst

**<font color=green > Question - 2 (10-*pts*)**
    
Now you have seen the cornerness definition of Harris on $Equation ~(12)$. Another relevant definition of cornerness is defined by  *[Shi and Tomasi](https://ieeexplore.ieee.org/document/323794)*, after the original definition of Harris. Check their algorithm and answer the following questions:
    
1. How do they define cornerness? Write down their definition using the notations of $Equation ~(12)$.
    

2. Does the Shi-Tomasi Corner Detector satisfy the following properties: translation invariance, rotation invariance, scale invariance? Explain your reasoning.
    

3. In the following scenarios, what could be the relative cornerness values assigned by Shi and Tomasi? Explain your reasoning.
    
    (a) Both eigenvalues are near 0.
    
    (b) One eigenvalue is big and the other is near zero.
    
    (c) Both eigenvalues are big.

*Q2 - Type your answers here*

## 2. Optical Flow - Lucas-Kanade Algorithm (35pts)

Optical flow is the apparent motion of image pixels or regions from one frame to the next, which results from moving objects in the image or from camera motion. Underlying optical flow is typically an assumption of $\textit{brightness constancy}$. That is the image values (brightness, color, etc) remain constant over time, though their 2D position in the image may change. Algorithms for estimating optical flow exploit this assumption in various ways to compute a velocity field that describes the horizontal and vertical motion of every pixel in the image. For a 2D+t dimensional case a voxel at location $(x, y, t)$ with intensity $I(x, y, t)$ will have moved by $\delta_x$, $\delta_y$ and $\delta_t$ between the two image frames, and the following image constraint equation can be given:

\begin{equation}
I(x,y,t) = I(x+\delta_x, y+\delta_y, t+\delta_t). \tag{15}
\end{equation}

Assuming the movement to be small, the image constraint at I(x, y, t) can be extended using Taylor series, truncated to first-order terms:

\begin{equation}
\tag{16}
I(x+\delta_x, y+\delta_y, t+\delta_t) = I(x,y,t)+\frac{\partial I}{\partial x}\delta_x +\frac{\partial I}{\partial y}\delta_y+\frac{\partial I}{\partial t}\delta_t
\end{equation}

Since we assume changes in the image can purely be attributed to movement, we will get:

\begin{equation}
\tag{17}
\frac{\partial I}{\partial x}\frac{\delta_x}{\delta_t} +   \frac{\partial I}{\partial y}\frac{\delta_y}{\delta_t} + \frac{\partial I}{\partial t}\frac{\delta_t}{\delta_t} = 0
\end{equation}

or

\begin{equation}
\tag{18}
I_x V_x + I_y V_y = -I_t,
\end{equation}

where $V_x$ and $V_y$ are the $x$ and $y$ components of the velocity or optical flow of $I(x,y,t)$. Further, $I_x$, $I_y$ and $I_t$ are the derivatives of the image at $(x, y, t)$ in the corresponding directions, which defines the main equation of optical flow.

Optical flow is difficult to compute for two main reasons. First, in image regions that are roughly homogeneous, the optical flow is ambiguous, because the brightness constancy assumption is satisfied by many different motions. Second, in real scenes, the assumption is violated at motion boundaries and by miscellaneous lighting, non-rigid motions, shadows, transparency, reflections, etc. To address the former, all optical flow methods make some sort of assumption about the spatial variation of the optical flow that is used to resolve the ambiguity. Those are just assumptions about the world which are approximate and consequently may lead to errors in the flow estimates. The latter problem can be addressed by making much richer but more complicated assumptions about the changing image brightness or, more commonly, using robust statistical methods which can deal with 'violations' of the brightness constancy assumption.

### Lucas-Kanade Algorithm

We will be implementing the [Lucas-Kanade method](https://www.ri.cmu.edu/pub_files/pub3/lucas_bruce_d_1981_2/lucas_bruce_d_1981_2.pdf) for Optical Flow estimation. This method assumes that the optical flow is essentially constant in a local neighborhood of the pixel under consideration. Therefore, the main equation of the optical flow can be assumed to hold for all pixels within a window centered at the pixel under consideration. Let's consider pixel $p$. Then, for all pixels around $p$, the local image flow vector $(V_x, V_y)$ must satisfy:

\begin{array}{c}
\tag{19}
  I_x(q_1)V_x + I_y(q_1)V_y = - I_t(q_1)\\
  I_x(q_2)V_x + I_y(q_2)V_y = - I_t(q_2)\\
  \vdots \\
  I_x(q_n)V_x + I_y(q_n)V_y = - I_t(q_n),\\
\end{array}

where $q_1$, $q_2$, ... $q_n$ are the pixels inside the window around $p$. $I_x(q_i)$, $I_y(q_i)$, $I_t(q_i)$ are the partial derivatives of the image $I$ with respect to position $x$, $y$ and time $t$, evaluated at the point $q_i$ and at the current time. 

These equations can be written in matrix to form $Av=b$, where

\begin{equation}
\tag{20}
A = \begin{bmatrix}
I_x(q_1) & I_y(q_1)\\
I_x(q_2) & I_y(q_2)\\
\vdots   & \vdots  \\
I_x(q_n) & I_y(q_n)
\end{bmatrix},
v = \begin{bmatrix}
V_x\\ V_y
\end{bmatrix}, \mbox{ and }
b = \begin{bmatrix}
-I_t(q_1)\\
-I_t(q_2)\\
\vdots   \\
-I_t(q_n)
\end{bmatrix}.
\end{equation}

This system has more equations than unknowns and thus it is usually over-determined. The Lucas-Kanade method obtains a compromise solution by the weighted-least-squares principle. Namely, it solves the $2\times 2$ system as

\begin{equation}
\tag{21}
A^T A v = A^T b
\end{equation}

or

\begin{equation}
v = (A^T A)^{-1} A^T b.
\label{eqA} \tag{22}
\end{equation}

**<font color=green > Question - 1 (30-*pts*)**
    


## 2. Implementing the Lucas-Kanade Algorithm

In the following we implement the Lucas Kanade Algorithm, and present the optical flow between two consecutive image frames for three various motives. We do so by 

1. estimating the gradient and the temporal derivative of the entire image.

2. Solving equation 22 using the psudo inverse of the matrix A and the time derivative b over non overlapping regions of the image defined by a window size og 15x15 pixels.

In [ ]:
## from PIL import Image
dim_window = 15

# Part 1 - Divide image int non-overlapping regions of 15 x 15 pixels 

def crop_image(idx, img, windowsize):
    """ Crop image to a multiplum of window size
    """
    if (img.shape[idx] % windowsize == 0):
        img = np.array(img)
        print(' Original shape of PIL image maintained ', img.shape, ' on axis ', str(idx))
    
    else:
        img = np.array(img)
        pix = img.shape[idx] % windowsize
        if idx == 1:
            img = img[:, 0:-pix]
        elif idx == 0:
            img = img[:-pix,:]
        print(' Original shape of PIL image cropped by ', pix, ' pixels on axiz ', str(idx), ' to new shape ', img.shape)
    return img

def read_image(path, windowsize):
    """ Return Normalized gray scale cv2 image files
    """
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.normalize(img.astype('float'), None, 0.0, 1.0, cv2.NORM_MINMAX)
    
    # Crop image in case of surpluss pixels
    img = crop_image(0, img, windowsize)
    img = crop_image(1, img, windowsize)
    
    return img



sphere1 = read_image('data/sphere1.ppm',dim_window)
sphere2 = read_image('data/sphere2.ppm', dim_window)

ball1 = read_image('data/basketball/001.jpg',dim_window)
ball2 = read_image('data/basketball/002.jpg', dim_window)

toy1 = read_image('data/toy/00000001.jpg',dim_window)
toy2 = read_image('data/toy/00000092.jpg', dim_window)

synth1  = read_image('data/synth1.pgm', dim_window)
synth2  = read_image('data/synth2.pgm', dim_window)

monkaa1 = read_image('data/monkaa1.png',dim_window)
monkaa2 = read_image('data/monkaa2.png', dim_window)


In [ ]:
# Part 2.i Compute A

def derivatives(image1, image2, windowsize):
    # Define derivative filters
    Gx = np.reshape(np.array([-1,0,1]), (1,3))
    Gy = np.transpose(Gx)
    
    # Define vectors for storing velocity per tile
    I_x_ = 𝚜𝚒𝚐𝚗𝚊𝚕.𝚌𝚘𝚗𝚟𝚘𝚕𝚟𝚎𝟸𝚍(image1, Gx, mode = 'same')
    I_y_ = 𝚜𝚒𝚐𝚗𝚊𝚕.𝚌𝚘𝚗𝚟𝚘𝚕𝚟𝚎𝟸𝚍(image2, Gy, mode = 'same')
    I_t_ = image1-image2
    tiles_I_x = np.array([I_x_[x:x+windowsize,y:y+windowsize] for x in range(0,I_x_.shape[0],windowsize) for y in range(0,I_x_.shape[1],windowsize)])
    tiles_I_y = np.array([I_y_[x:x+windowsize,y:y+windowsize] for x in range(0,I_y_.shape[0],windowsize) for y in range(0,I_y_.shape[1],windowsize)])
    tiles_I_t = np.array([I_t_[x:x+windowsize,y:y+windowsize] for x in range(0,I_t_.shape[0],windowsize) for y in range(0,I_t_.shape[1],windowsize)])
    
    return tiles_I_x, tiles_I_y, tiles_I_t

def compute_velocities(windowsize, image1, image2):
    # Compute A and B
    I_x_, I_y_, I_t_ = derivatives(image1, image2, windowsize)
    vx, vy = [], []
    
    for i in range(len(I_x_)):
        # Temp store A, b
        A, b = [], []
        
        # Take the convolution with the derivative filter
        I_x = I_x_[i]
        I_y = I_y_[i]
        I_t = I_t_[i]
        
        # Calculate matrix A and vector b
        for n in range(windowsize):
            for m in range(windowsize):
                A.append([I_x[n,m], I_y[n,m]])
                b.append(I_t[n, m])
                    
        A = np.array(A)
        b = np.array(b)

        # Estimate Optical Flow by eq 22.
        v = np.matmul(np.linalg.pinv(A), b)

        vx.append(v[0]); vy.append(v[1])
            
    # Transform to 2D matrix
    xdim = int(image1.shape[0]/dim_window)
    ydim = int(image1.shape[1]/dim_window)
    vy_mat, vx_mat = np.zeros((xdim,ydim)), np.zeros((xdim, ydim))
    for j in range(xdim):
        vx_mat[j,:] = np.array(vx[j*ydim:(j+1)*ydim]).flatten()
        vy_mat[j,:] = np.array(vy[j*ydim:(j+1)*ydim]).flatten()
    
    return vx_mat, vy_mat


def quiverplot(image1, image2, dim_window, title, scalevalue,image=False):
    # Compute velocity matrix for two subsequent images
    vx_,vy_ = compute_velocities(dim_window, image1, image2)
    
    #Plot quiver plot
    plt.title(title)

    x, y = np.meshgrid(np.arange(0,int(image1.shape[1]/dim_window),1), np.arange(0,int(image1.shape[0]/dim_window),1))
    plt.imshow(image1, alpha = 0.3)
    plt.quiver(x*dim_window + dim_window/2, y*dim_window + dim_window/2, -vx_, -vy_, scale = scalevalue, angles = 'xy', color='black')
    plt.figtext(0.5, 0.01, 'Optical flow estimation using the Lucas-Kanade method', wrap=True, horizontalalignment='center', fontsize=12)
    plt.xticks(fontsize = 12); plt.yticks(fontsize = 12)
    if image:
        xdim = int(image1.shape[0]/dim_window)
        ydim = int(image1.shape[1]/dim_window)
        imgC_array = np.array(image1)
        tiles = np.array([imgC_array[x:x+dim_window,y:y+dim_window] for x in range(0,imgC_array.shape[0],dim_window) for y in range(0,imgC_array.shape[1],dim_window)])
        plt.figure()
        for i in range(int(xdim*ydim)):  
                plt.subplot(xdim, ydim, i +1)
                plt.imshow(tiles[i], cmap = 'gray')
                plt.xticks([]);plt.yticks([])
        plt.suptitle('Image separation: '+ str(xdim)+ 'x'+ str(ydim)+ ' tiles comprised of '+ str(dim_window)+ 'x'+ str(dim_window)+ ' pixels')
        plt.show()
        
    plt.show()
    return 0

quiverplot(sphere1, sphere2, dim_window, 'Figure 2.1: Optical flow of Sphere', 30, image = True)
quiverplot(synth1, synth2, dim_window, 'Figure 2.2: Optical flow of Synth', 10, image = True)
quiverplot(monkaa1, monkaa2, dim_window, 'Figure 2.3: Optical flow of Monkaa', 25, image = True)


**<font color=green > Question - 2 (5-*pts*)**

# 2.2 Lucas-Kanade vs. Horn-Schunck method 

The Lucas Kanade (LK) and the Horn-Schunck (HS) methods both aim to estimate the optical flow between frames, but differ in their ground assumptions and approaches towards calculating the optical flow vectors. 

## 2.2.1 Global and Local methods

The LK method assumes that flow is constant in a local neighborhood, and as such assumes that the optical flow equation (Eq. 18) holds for all pixels in that local region (Eq. 21). The optical flow equation is then solved by the least squares criterion (Eq. 22) [1]. In this sense, the LK approach operates at a local scale, as it only utilize information in the neighborhood defined by a fixed size tile. In particular, LK estimates the average motion of such a tile using the tiles' spatial intensity gradient information and the time derivative between the consecutive frames. However, exactly how 'local' the method can claim to be depends on the size of one tile and the total image size relative to the motion depicted by the frames.


The Horn-Schunck method is on the other hand a global method that assumes that the apparent velocity of the brightness pattern varies smoothly everywhere in the image. After imposing this global smoothness constraint, the method tries to minimize a global energy function expressed as the full spatial integral over all the derivatives of the intensity in every pixel, weighted with the optical flow vector and a regularization constant. The energy function is then minimized by solving the corresponding set of multi-dimensional Euler-Lagrange equations [2]. The velocity components of each pixel that define the optical flow vector which minimizes the energy function is selected as estimators. In this sense, the HS method is truly global in that the optical velocities at each pixel is estimated using the joint information from all pixels in the frames simultaneously. 

## 2.2.2 Behavior in flat regions 
The global method (HS) and more local approach (LK) differ in their behavior in flat regions. A flat region is a region where all the derivatives are (approximately) zero, i.e. homogeneous image patches with approximately constant intensity. 

When the flow vectors are estimated using LK, a flat region across two consecutive image frames will suffer from the aperture problem. As the method only consider local information with a fixed window size it becomes very hard to detect wether there are no motion between two consecutive frames in a flat region, or wether the flat patch is part of a feature in motion which range is exceeding the neighborhood. The latter can yield a net zero result for the local flow vector, while in fact it should have had the same value as its' neighboring flow vectors where motion can be detected due to the presence of a boundary. This is illustrated in the calculations above. For the sphere and the synth motion the LK does a decent job of finding the flow vectors. It can be seen by looking at the tile separation illustrations for these two motives that there are few flat regions used in the estimations. Given that thethe texture of the sphere and the synth varies to a relative high degree it is almost always possible to measure some variation in the intensity. In the case of the Monkaa motive the situation is more complicated, as the presence of flat regions appear to be higher for some areas, and the relative motion quite drastic in other regions.

If the area does not contain any of the edges perpendicular to the movement, the gradient will be singular, and you will not be able to estimate the optical flow. The HS method fixes the aperture problem, as its global methodology minimizes the energy function in all pixels simultaneously, as such all flow vectors contain information about the entire environment of the image, and not just that which is contained in a local neighborhood. There is no risk of excluding a boundary by selecting the wrong window size of selecting a neighborhood containing only the flat region or a feature that in fact is in motion between the frames. However, such a method is also more sensitive to noise and relative motion differences than local methods like the LK, due to the impact of the global nature of the methodology on the final results of all flow vectors. 

*In conclusion*, while the LK method is likely a good and sufficient approach for motives such as the sphere and the synth, it is probably a better idea to use global methods such as HK when handling motives like monkaa where the texture regions and relative motions between neighborhoods are more complicated. 


### References Section 2 

[1] Lucas, Bruce & Kanade, Takeo. (1981). An Iterative Image Registration Technique with an Application to Stereo Vision (IJCAI). [No source information available]. 81. 


[2] Berthold K.P. Horn, Brian G. Schunck,
Determining optical flow,
Artificial Intelligence,
Volume 17, Issues 1–3,
1981,
Pages 185-203,
ISSN 0004-3702,
https://doi.org/10.1016/0004-3702(81)90024-2.
(https://www.sciencedirect.com/science/article/pii/0004370281900242)

## 3. Feature Tracking (20-pts)

In this part of the assignment, you will implement a simple feature-tracking algorithm. The aim is to extract visual features, like corners, and track them over multiple frames.

**<font color=green > Question - 1 (18-*pts*)**

1. Implement a simple feature-tracking algorithm by following below steps.
    
    (a) Locate feature points on the first frame by using the Harris Corner Detector, that you implemented in Section 1. 
    
    (b) Track these points using the Lucas-Kanade algorithm for optical flow estimation, that you implemented in the Section 2.
    
    
2. Prepare a video for each sample image sequences. These videos should visualize the initial feature points and the optical flow. Test your implementation and prepare visualization videos for **basketball** and **toy** samples.

Include a demo function to run your code.

In [ ]:
import matplotlib.animation as animation

# Q1 - Type your code here
def output(folder):
    image_dir = './data/' + folder +'/'
    files = os.listdir(image_dir)
    n_files = len(files)
    
    
    im_1 = cv2.imread(os.path.join(image_dir, files[0]))
    im_1 = cv2.cvtColor(im_1, cv2.COLOR_BGR2GRAY)
    [h,w] = im_1.shape
    dim_window = 15
    x_block = w // 15 
    y_block = h // 15 
    I_x, I_y = calc_derivatives(im_1)
    _,points = harris_corner_detector(I_x, I_y, max_filter_n=5, threshold=5e6)
    fig = visualize_corners(im_1, points, title='', alpha=0.7)
    
    for i in range(n_files-1):
        print('IMAGE', i)
        im_start = read_image(os.path.join(image_dir, files[i]), dim_window)
        im_next = read_image(os.path.join(image_dir, files[i+1]), dim_window)

        
        divided_im1 = divide_image(im_start, 15,plot=False)
        divided_im2 = divide_image(im_next,15,plot=False)
        vx_,vy_ = compute_velocities(divided_im1, divided_im2, dim_window,im_start,im_next)
        
        start_idx = 0
        vy_list = []
        vx_list = []
        points = points.astype(np.float32)

        for point in points:
            c_idx = int(np.floor(point[1]/15))
            r_idx = int(np.floor((h-point[0])/15))
            '''
            visualize_corners(im_start, [point], title='')
            '''
            #data = data.reshape(fig.canvas.get_width_height()[::-1] + (3,))

            if r_idx >= y_block:
                r_idx = y_block-1
            if c_idx >= x_block:
                c_idx = x_block-1
            vy_list.append(vy_[r_idx,c_idx])
            vx_list.append(vx_[r_idx,c_idx] )
            '''

            plt.figure()
            quiverplot(divided_im1, divided_im2, 15, '', scalevalue=50,image=False,image1=im_start, image2 = im_next,c_idx=c_idx,r_idx=r_idx)
            
            print('Old point ', point[0],point[1])
            print(- vy_[r_idx,c_idx], vx_[r_idx,c_idx] )
            '''

            points[start_idx][0] = point[0] - vy_[r_idx,c_idx] 
            points[start_idx][1] = point[1] + vx_[r_idx,c_idx] 
            '''

            print('New point ', type(points[start_idx][0]),points[start_idx][1])
            '''

            if points[start_idx][0] < 0 : 
                points[start_idx][0] = 0
            
            if points[start_idx][1] < 0 : 
                points[start_idx][1] = 0
                
            start_idx += 1
            '''
            quiverplot(divided_im1, divided_im2, 15, '', scalevalue=50,image=False,image1=im_start, image2 = im_next,c_idx= int(np.floor(point[1]/15)),r_idx=int(np.floor((h-point[0])/15)))
            pas = input("Next: ")
            '''

        fig = visualize_corners(im_start, points, title='', alpha=0.7,save=True,name='{}'.format(i+100),show = False,show_ax=False,quiver=True,point=points,vx_list=vx_list,vy_list=vy_list,folder=folder)
    image_dir = './Results/' + folder + '/'
    files = os.listdir(image_dir)
    file = sorted(files)
    n_files = len(files)
    im_out = []
    fig = plt.figure()
    
    for i in range(n_files):
        im = cv2.imread(os.path.join(image_dir, files[i]))
        im_out.append([plt.imshow(im,animated=True)])
    ani = animation.ArtistAnimation(fig, im_out)
    plt.axis('off')
    ani.save(folder+ '.mp4')
        

    plt.show()
    

In [ ]:
output('toy')

*Q1 - Type your answers here*

**<font color=green > Question - 2 (2-*pts*)**

Why do we need feature tracking even though we can detect features for each and every frame? 

If a particular feature is hidden from the image's view (i.e an object passes behind another object), that particular feature will not be detected in the next frame and subsequently that feature will be lost. By tracking the feature  